<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/qwen_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets>=2.18.0 transformers>=4.38.2 sentence-transformers>=2.5.1 setfit>=1.0.3 accelerate>=0.27.2 seqeval>=1.2.2

In [ ]:
model_path = 'Qwen/Qwen2.5-0.5B'
output_dir = '/content/drive/My Drive/AiExpertCource/project/Qwen/Qwen_tag'

In [ ]:
!pip install datasets
!pip install scikit-multilearn
!pip install peft
!pip install transformers
!pip install bitsandbytes
!pip install evaluate
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login

# 로그인 함수 호출
login()

In [ ]:
import os
import random
import functools
import csv
import json
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import load_dataset, Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
import ast
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_training_samples.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_validation_samples.csv',
    split='train'
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = DatasetDict({
    'train': Dataset.from_dict({'Title': dataset_train['Title'], 'Body': dataset_train['Body'], 'Tags_new': dataset_train['Tags_new']}),
    'val': Dataset.from_dict({'Title': dataset_valid['Title'], 'Body': dataset_valid['Body'], 'Tags_new': dataset_valid['Tags_new']}),
})

In [ ]:
classes = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
class2id = {'Algorithms' :0, 'Backend' : 1, 'Data Science' : 2, 'Databases' : 3, 'Dev Tools' : 4, 'Frontend' : 5, 'Mobile' :6, 'Systems' : 7, 'iOS/macOS' : 8}
id2class = {0 : 'Algorithms', 1: 'Backend', 2 : 'Data Science', 3 : 'Databases', 4 : 'Dev Tools', 5 : 'Frontend', 6 : 'Mobile', 7 : 'Systems', 8 :'iOS/macOS'}

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_path)
# 패딩 토큰을 종료 토큰으로 설정
tokenizer.pad_token = tokenizer.eos_token

# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
  # 'Tags_new'에서 라벨을 파싱
   all_labels =  ast.literal_eval(examples['Tags_new'])
   # 클래스 수만큼 0으로 초기화된 라벨 리스트 생성
   labels = [0. for i in range(len(classes))]
   # 해당 라벨에 1로 마킹
   for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.

   tokenized_inputs = tokenizer(examples['Title'] + ' ' + examples['Body'])
   tokenized_inputs['labels'] = labels
   return tokenized_inputs

tokenized_ds = dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer))
tokenized_ds = tokenized_ds.with_format('torch')

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

In [ ]:
tokenized_ds = tokenized_ds.remove_columns(["Title", "Body", "Tags_new"])

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6500
    })
})

In [ ]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d


# define which metrics to compute for evaluation
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

  predictions, labels = eval_pred
  predictions = sigmoid(predictions)
  predictions = (predictions > 0.5).astype(int)
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1_score_result, _ = precision_recall_fscore_support(labels, predictions, average='micro')

  flat_predictions = predictions.reshape(-1)
  flat_labels = labels.reshape(-1)
  flat_accuracy = accuracy_score(flat_labels, flat_predictions)

  return {
        'flat_accuracy' : flat_accuracy,
        'accuracy' : accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score_result
        }


# create custom trainer class to be able to pass label weights and calculate mutilabel loss
class CustomTrainer(Trainer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32))
        return (loss, outputs) if return_outputs else loss

In [ ]:
# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

# load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    quantization_config=quantization_config,
    num_labels=len(classes)
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# define training args
training_args = TrainingArguments(
    output_dir = "Qwen_tag",
    learning_rate = 1e-4,
    per_device_train_batch_size = 1,  # further reduce batch size to 1
    per_device_eval_batch_size = 1,  # adjust eval batch size accordingly
    num_train_epochs = 5,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
    gradient_accumulation_steps = 4,  # increase gradient accumulation steps
    # fp16=True, # Try mixed precision training, if supported by your GPU.
    # gradient_checkpointing=True # Enable gradient checkpointing to save memory
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['val'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics
)



NameError: name 'TrainingArguments' is not defined

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Flat Accuracy,Accuracy,Precision,Recall,F1 Score
1,0.133800,0.104709,0.963368,0.728462,0.849945,0.884723,0.866985
2,0.092500,0.119692,0.961333,0.708615,0.828742,0.899291,0.862576
3,0.069700,0.139991,0.961162,0.712615,0.830862,0.894223,0.861379
4,0.044600,0.171837,0.959692,0.701538,0.822987,0.893463,0.856778
5,0.031100,0.189598,0.959607,0.704154,0.827085,0.885863,0.855465


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommend

TrainOutput(global_step=25000, training_loss=0.07612948265075684, metrics={'train_runtime': 40952.651, 'train_samples_per_second': 2.442, 'train_steps_per_second': 0.61, 'total_flos': 6.22744405129728e+16, 'train_loss': 0.07612948265075684, 'epoch': 5.0})

In [2]:
# 모델을 평가 모드로 설정
model.eval()

NameError: name 'model' is not defined

In [ ]:
# 예측 함수 정의
def predict(texts, m, tk):
    # 텍스트를 토큰화하고 텐서로 변환
    inputs = tk(texts, padding='max_length', truncation=True, max_length=2048, return_tensors='pt')

    # 모델을 사용해 예측 수행
    with torch.no_grad():
        outputs = m(**inputs)
        logits = outputs.logits

    # 시그모이드를 사용해 확률로 변환
    probabilities = torch.sigmoid(logits).numpy()

    # 각 클래스에 대해 threshold를 0.5로 설정하여 예측값(0 또는 1)으로 변환
    predictions = (probabilities > 0.5).astype(int)

    return predictions, probabilities

In [ ]:
# 샘플 텍스트 입력 및 예측 수행
sample_texts = [
  "FFmpeg hevc rtsp stream decoding with frame loss" +
  "I need to decode my rtsp stream. When i`m using default HEVC and have some corrupted frames, my screen looks like this: Corrupted frame using HEVC, Grey image. But instead of this, I want to have a corrupted picture with pixels issue, like this: Same corrupted frame, but using HEVC_QSV."
]


In [ ]:
def print_tag(prds):
  arr = np.array(prds[0])
  # 값이 1인 인덱스에 해당하는 매핑 가져오기
  indices = np.where(arr == 1)[0]
  indices
  for idx in indices:
    tag = id2class[idx]
    print(tag)

In [ ]:
# 예측 수행
predictions, probabilities = predict(sample_texts, model, tokenizer)

In [ ]:
predictions

In [ ]:
probabilities

In [ ]:
print_tag(predictions)

In [ ]:
# 3. LoRA와 base 모델 병합 후 저장
model = model.merge_and_unload()  # LoRA를 base 모델에 병합
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
new_model = AutoModelForSequenceClassification.from_pretrained(
    output_dir,
    num_labels=len(classes)
)

In [ ]:
new_model.eval()

In [ ]:
# 예측 수행
predictions, probabilities = predict(sample_texts, new_model, tokenizer)

In [ ]:
print_tag(predictions)